In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import os

# Set up visualization style
plt.style.use('default')
sns.set_palette("husl")

# Create output directory for graphs
output_dir = r"D:\Wellbeing Survey Project\Graphs"
os.makedirs(output_dir, exist_ok=True)

def load_and_clean_data(file_path):
    """Load and clean the survey data"""
    print("Loading data...")
    df = pd.read_excel(file_path, sheet_name='Sheet1')
    
    # Clean column names
    df.columns = df.columns.str.strip()
    
    # Define numeric columns
    numeric_cols = [
        'What grade level are you?', 
        'How often do you feel positive in school?', 
        'How often do you feel supported by your teachers and advisors?', 
        'How often do you feel listened to by your teachers and advisors?',
        'How often do you feel listened to and supported by your friends?',
        'How do you feel you are managing the workload?',
        'To what extent do you feel the workload is making you feel stressed?',
        'Overall, how do you feel about the first few months of school?'
    ]
    
    # Convert to numeric, handling errors
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Clean SchoolLevel column
    df['SchoolLevel'] = df['SchoolLevel'].str.strip()
    
    # Extract date from timestamp
    df['Date'] = pd.to_datetime(df['Timestamp']).dt.date
    
    print(f"Data loaded successfully. Shape: {df.shape}")
    return df, numeric_cols

def create_summary_statistics(df, numeric_cols):
    """Create summary statistics"""
    print("\nGenerating summary statistics...")
    
    # Summary by school level
    school_summary = df.groupby('SchoolLevel')[numeric_cols].agg(['mean', 'median', 'count'])
    
    # Summary by term
    term_summary = df.groupby('Term')[numeric_cols].agg(['mean', 'median', 'count'])
    
    # Save summaries to Excel
    with pd.ExcelWriter(os.path.join(output_dir, 'summary_statistics.xlsx')) as writer:
        school_summary.to_excel(writer, sheet_name='By School Level')
        term_summary.to_excel(writer, sheet_name='By Term')
    
    return school_summary, term_summary

def plot_school_level_comparison(df, numeric_cols):
    """Create comparison charts by school level"""
    print("Creating school level comparison charts...")
    
    # Set up the figure
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.ravel()
    
    # Plot each metric
    for i, col in enumerate(numeric_cols):
        # Calculate means
        means = df.groupby('SchoolLevel')[col].mean()
        
        # Create bar plot
        axes[i].bar(means.index, means.values, alpha=0.7)
        axes[i].set_title(f'Average {col}')
        axes[i].set_ylabel('Score (1-10)')
        axes[i].tick_params(axis='x', rotation=45)
        
        # Add value labels on bars
        for j, v in enumerate(means.values):
            axes[i].text(j, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'school_level_comparison.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_term_comparison(df, numeric_cols):
    """Create comparison charts by term"""
    print("Creating term comparison charts...")
    
    # Set up the figure
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.ravel()
    
    # Plot each metric
    for i, col in enumerate(numeric_cols):
        # Calculate means
        means = df.groupby('Term')[col].mean()
        
        # Create bar plot
        axes[i].bar(means.index, means.values, alpha=0.7)
        axes[i].set_title(f'Average {col}')
        axes[i].set_ylabel('Score (1-10)')
        axes[i].tick_params(axis='x', rotation=45)
        
        # Add value labels on bars
        for j, v in enumerate(means.values):
            axes[i].text(j, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'term_comparison.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_correlation_heatmap(df, numeric_cols):
    """Create correlation heatmap"""
    print("Creating correlation heatmap...")
    
    plt.figure(figsize=(12, 10))
    correlation_matrix = df[numeric_cols].corr()
    
    # Create heatmap
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    
    plt.title('Correlation Between Wellbeing Metrics')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'correlation_heatmap.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_stress_analysis(df):
    """Analyze stress levels"""
    print("Creating stress analysis charts...")
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Stress by school level
    stress_by_school = df.groupby('SchoolLevel')['To what extent do you feel the workload is making you feel stressed?'].mean()
    axes[0].bar(stress_by_school.index, stress_by_school.values, alpha=0.7, color='lightcoral')
    axes[0].set_title('Average Stress Level by School Level')
    axes[0].set_ylabel('Stress Level (1-10)')
    
    # Add value labels
    for i, v in enumerate(stress_by_school.values):
        axes[0].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    # Stress by term
    stress_by_term = df.groupby('Term')['To what extent do you feel the workload is making you feel stressed?'].mean()
    axes[1].bar(stress_by_term.index, stress_by_term.values, alpha=0.7, color='lightcoral')
    axes[1].set_title('Average Stress Level by Term')
    axes[1].set_ylabel('Stress Level (1-10)')
    
    # Add value labels
    for i, v in enumerate(stress_by_term.values):
        axes[1].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'stress_analysis.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_positive_feelings_analysis(df):
    """Analyze positive feelings"""
    print("Creating positivity analysis charts...")
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Positivity by school level
    pos_by_school = df.groupby('SchoolLevel')['How often do you feel positive in school?'].mean()
    axes[0].bar(pos_by_school.index, pos_by_school.values, alpha=0.7, color='lightgreen')
    axes[0].set_title('Average Positivity Level by School Level')
    axes[0].set_ylabel('Positivity Level (1-10)')
    
    # Add value labels
    for i, v in enumerate(pos_by_school.values):
        axes[0].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    # Positivity by term
    pos_by_term = df.groupby('Term')['How often do you feel positive in school?'].mean()
    axes[1].bar(pos_by_term.index, pos_by_term.values, alpha=0.7, color='lightgreen')
    axes[1].set_title('Average Positivity Level by Term')
    axes[1].set_ylabel('Positivity Level (1-10)')
    
    # Add value labels
    for i, v in enumerate(pos_by_term.values):
        axes[1].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'positivity_analysis.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_workload_management(df):
    """Analyze workload management"""
    print("Creating workload management charts...")
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Workload management by school level
    workload_by_school = df.groupby('SchoolLevel')['How do you feel you are managing the workload?'].mean()
    axes[0].bar(workload_by_school.index, workload_by_school.values, alpha=0.7, color='lightblue')
    axes[0].set_title('Workload Management by School Level')
    axes[0].set_ylabel('Management Score (1-10)')
    
    # Add value labels
    for i, v in enumerate(workload_by_school.values):
        axes[0].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    # Workload management by term
    workload_by_term = df.groupby('Term')['How do you feel you are managing the workload?'].mean()
    axes[1].bar(workload_by_term.index, workload_by_term.values, alpha=0.7, color='lightblue')
    axes[1].set_title('Workload Management by Term')
    axes[1].set_ylabel('Management Score (1-10)')
    
    # Add value labels
    for i, v in enumerate(workload_by_term.values):
        axes[1].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'workload_management.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_support_analysis(df):
    """Analyze support systems"""
    print("Creating support analysis charts...")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Teacher support by school level
    teacher_support = df.groupby('SchoolLevel')['How often do you feel supported by your teachers and advisors?'].mean()
    axes[0, 0].bar(teacher_support.index, teacher_support.values, alpha=0.7, color='orange')
    axes[0, 0].set_title('Teacher Support by School Level')
    axes[0, 0].set_ylabel('Support Score (1-10)')
    
    # Add value labels
    for i, v in enumerate(teacher_support.values):
        axes[0, 0].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    # Teacher support by term
    teacher_support_term = df.groupby('Term')['How often do you feel supported by your teachers and advisors?'].mean()
    axes[0, 1].bar(teacher_support_term.index, teacher_support_term.values, alpha=0.7, color='orange')
    axes[0, 1].set_title('Teacher Support by Term')
    axes[0, 1].set_ylabel('Support Score (1-10)')
    
    # Add value labels
    for i, v in enumerate(teacher_support_term.values):
        axes[0, 1].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    # Friend support by school level
    friend_support = df.groupby('SchoolLevel')['How often do you feel listened to and supported by your friends?'].mean()
    axes[1, 0].bar(friend_support.index, friend_support.values, alpha=0.7, color='purple')
    axes[1, 0].set_title('Friend Support by School Level')
    axes[1, 0].set_ylabel('Support Score (1-10)')
    
    # Add value labels
    for i, v in enumerate(friend_support.values):
        axes[1, 0].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    # Friend support by term
    friend_support_term = df.groupby('Term')['How often do you feel listened to and supported by your friends?'].mean()
    axes[1, 1].bar(friend_support_term.index, friend_support_term.values, alpha=0.7, color='purple')
    axes[1, 1].set_title('Friend Support by Term')
    axes[1, 1].set_ylabel('Support Score (1-10)')
    
    # Add value labels
    for i, v in enumerate(friend_support_term.values):
        axes[1, 1].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'support_analysis.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_overall_sentiment(df):
    """Analyze overall sentiment"""
    print("Creating overall sentiment charts...")
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Overall sentiment by school level
    sentiment_by_school = df.groupby('SchoolLevel')['Overall, how do you feel about the first few months of school?'].mean()
    axes[0].bar(sentiment_by_school.index, sentiment_by_school.values, alpha=0.7, color='gold')
    axes[0].set_title('Overall Sentiment by School Level')
    axes[0].set_ylabel('Sentiment Score (1-10)')
    
    # Add value labels
    for i, v in enumerate(sentiment_by_school.values):
        axes[0].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    # Overall sentiment by term
    sentiment_by_term = df.groupby('Term')['Overall, how do you feel about the first few months of school?'].mean()
    axes[1].bar(sentiment_by_term.index, sentiment_by_term.values, alpha=0.7, color='gold')
    axes[1].set_title('Overall Sentiment by Term')
    axes[1].set_ylabel('Sentiment Score (1-10)')
    
    # Add value labels
    for i, v in enumerate(sentiment_by_term.values):
        axes[1].text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'overall_sentiment.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_response_trends(df):
    """Plot response trends over time"""
    print("Creating response trend charts...")
    
    # Group by date and school level
    daily_responses = df.groupby(['Date', 'SchoolLevel']).size().unstack(fill_value=0)
    
    plt.figure(figsize=(12, 6))
    daily_responses.plot(kind='line', marker='o')
    plt.title('Survey Responses Over Time')
    plt.ylabel('Number of Responses')
    plt.xlabel('Date')
    plt.legend(title='School Level')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'response_trends.png'), dpi=300, bbox_inches='tight')
    plt.close()

def main():
    """Main function to run the analysis"""
    file_path = r"D:\Wellbeing Survey Project\combined_terms.xlsx"
    
    # Load and clean data
    df, numeric_cols = load_and_clean_data(file_path)
    
    # Create summary statistics
    school_summary, term_summary = create_summary_statistics(df, numeric_cols)
    
    # Create all visualizations
    plot_school_level_comparison(df, numeric_cols)
    plot_term_comparison(df, numeric_cols)
    plot_correlation_heatmap(df, numeric_cols)
    plot_stress_analysis(df)
    plot_positive_feelings_analysis(df)
    plot_workload_management(df)
    plot_support_analysis(df)
    plot_overall_sentiment(df)
    plot_response_trends(df)
    
    print(f"\nAnalysis complete! All graphs have been saved to: {output_dir}")

if __name__ == "__main__":
    main()

Loading data...
Data loaded successfully. Shape: (480, 14)

Generating summary statistics...
Creating school level comparison charts...


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Creating term comparison charts...
Creating correlation heatmap...
Creating stress analysis charts...


posx and posy should be finite values


Creating positivity analysis charts...


...

posx and posy should be finite values
posx and posy should be finite values


Creating support analysis charts...


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Creating overall sentiment charts...
Creating response trend charts...

Analysis complete! All graphs have been saved to: D:\Wellbeing Survey Project\Graphs


<Figure size 1200x600 with 0 Axes>

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.ticker import MaxNLocator

# Set up the style
plt.style.use('default')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 10

# Create output directory
output_dir = r"D:\Wellbeing Survey Project\Summary_Graphs"
os.makedirs(output_dir, exist_ok=True)

def create_visualizations():
    """Create visualizations from the summary statistics"""
    
    # Load the summary data
    summary_path = r"D:\Wellbeing Survey Project\Graphs\summary_statistics.xlsx"
    
    # School level data - handle multi-index columns
    school_df = pd.read_excel(summary_path, sheet_name='By School Level', header=[0, 1])
    
    # Flatten the multi-index columns
    school_df.columns = ['_'.join(col).strip('_') for col in school_df.columns.values]
    
    # Rename the first column to 'SchoolLevel'
    school_df = school_df.rename(columns={school_df.columns[0]: 'SchoolLevel'})
    
    # Term data - handle multi-index columns
    term_df = pd.read_excel(summary_path, sheet_name='By Term', header=[0, 1])
    
    # Flatten the multi-index columns
    term_df.columns = ['_'.join(col).strip('_') for col in term_df.columns.values]
    
    # Rename the first column to 'Term'
    term_df = term_df.rename(columns={term_df.columns[0]: 'Term'})
    
    # Clean up the data
    school_df = school_df[school_df['SchoolLevel'].isin(['High School', 'Middle School'])]
    
    # Metrics to visualize
    metrics = [
        'What grade level are you?_mean',
        'How often do you feel positive in school?_mean',
        'How often do you feel supported by your teachers and advisors?_mean',
        'How often do you feel listened to by your teachers and advisors?_mean',
        'How often do you feel listened to and supported by your friends?_mean',
        'How do you feel you are managing the workload?_mean',
        'To what extent do you feel the workload is making you feel stressed?_mean',
        'Overall, how do you feel about the first few months of school?_mean'
    ]
    
    # Clean metric names for display
    clean_metric_names = {
        'What grade level are you?_mean': 'Grade Level',
        'How often do you feel positive in school?_mean': 'Positivity in School',
        'How often do you feel supported by your teachers and advisors?_mean': 'Teacher Support',
        'How often do you feel listened to by your teachers and advisors?_mean': 'Feeling Listened To (Teachers)',
        'How often do you feel listened to and supported by your friends?_mean': 'Friend Support',
        'How do you feel you are managing the workload?_mean': 'Workload Management',
        'To what extent do you feel the workload is making you feel stressed?_mean': 'Stress Level',
        'Overall, how do you feel about the first few months of school?_mean': 'Overall School Experience'
    }
    
    # 1. School Level Comparison Radar Chart
    print("Creating radar chart...")
    create_radar_chart(school_df, metrics, clean_metric_names)
    
    # 2. School Level Bar Charts
    print("Creating school level comparison charts...")
    create_school_comparison_charts(school_df, metrics, clean_metric_names)
    
    # 3. Term Comparison Charts
    print("Creating term comparison charts...")
    create_term_comparison_charts(term_df, metrics, clean_metric_names)
    
    # 4. Response Count Charts
    print("Creating response count charts...")
    create_response_count_charts(school_df, term_df)
    
    # 5. Detailed Analysis Charts
    print("Creating detailed analysis charts...")
    create_detailed_analysis_charts(school_df, term_df, metrics, clean_metric_names)
    
    print(f"All visualizations saved to: {output_dir}")

def create_radar_chart(school_df, metrics, clean_metric_names):
    """Create a radar chart comparing High School vs Middle School"""
    
    # Filter data
    hs_data = school_df[school_df['SchoolLevel'] == 'High School'].iloc[0]
    ms_data = school_df[school_df['SchoolLevel'] == 'Middle School'].iloc[0]
    
    # Prepare data for radar chart
    categories = [clean_metric_names[metric] for metric in metrics]
    N = len(categories)
    
    # Values for each category
    hs_values = [hs_data[metric] for metric in metrics]
    ms_values = [ms_data[metric] for metric in metrics]
    
    # What will be the angle of each axis in the plot
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Complete the circle
    
    # Initialise the spider plot
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    
    # Draw one axe per variable + add labels
    plt.xticks(angles[:-1], categories, size=12)
    
    # Plot data
    hs_values += hs_values[:1]
    ms_values += ms_values[:1]
    
    # Draw the plot
    ax.plot(angles, hs_values, linewidth=2, linestyle='solid', label='High School', color='#FF6B6B')
    ax.fill(angles, hs_values, alpha=0.25, color='#FF6B6B')
    
    ax.plot(angles, ms_values, linewidth=2, linestyle='solid', label='Middle School', color='#4ECDC4')
    ax.fill(angles, ms_values, alpha=0.25, color='#4ECDC4')
    
    # Add a title
    plt.title('Wellbeing Metrics Comparison: High School vs Middle School', size=15, y=1.1)
    ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    
    # Save the chart
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'radar_chart_comparison.png'), dpi=300, bbox_inches='tight')
    plt.close()

def create_school_comparison_charts(school_df, metrics, clean_metric_names):
    """Create bar charts comparing High School and Middle School"""
    
    # Filter data
    hs_data = school_df[school_df['SchoolLevel'] == 'High School'].iloc[0]
    ms_data = school_df[school_df['SchoolLevel'] == 'Middle School'].iloc[0]
    
    # Create a figure with subplots
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.ravel()
    
    colors = ['#FF6B6B', '#4ECDC4']  # High School, Middle School
    
    for i, metric in enumerate(metrics):
        hs_value = hs_data[metric]
        ms_value = ms_data[metric]
        
        # Create bar chart
        bars = axes[i].bar(['High School', 'Middle School'], [hs_value, ms_value], 
                          color=colors, alpha=0.7)
        
        # Add value labels on bars
        for bar, value in zip(bars, [hs_value, ms_value]):
            height = bar.get_height()
            axes[i].text(bar.get_x() + bar.get_width()/2., height + 0.05,
                        f'{value:.2f}', ha='center', va='bottom')
        
        axes[i].set_title(clean_metric_names[metric])
        axes[i].set_ylabel('Score (1-10)')
        axes[i].set_ylim(0, 10)
        axes[i].yaxis.set_major_locator(MaxNLocator(integer=True))
    
    plt.suptitle('Wellbeing Metrics: High School vs Middle School Comparison', fontsize=16)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'school_comparison_detailed.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a combined bar chart
    plt.figure(figsize=(14, 8))
    x = np.arange(len(metrics))
    width = 0.35
    
    hs_values = [hs_data[metric] for metric in metrics]
    ms_values = [ms_data[metric] for metric in metrics]
    
    plt.bar(x - width/2, hs_values, width, label='High School', alpha=0.7, color='#FF6B6B')
    plt.bar(x + width/2, ms_values, width, label='Middle School', alpha=0.7, color='#4ECDC4')
    
    plt.xlabel('Metrics')
    plt.ylabel('Average Score')
    plt.title('Wellbeing Metrics Comparison: High School vs Middle School')
    plt.xticks(x, [clean_metric_names[metric] for metric in metrics], rotation=45, ha='right')
    plt.legend()
    plt.ylim(0, 10)
    plt.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'school_comparison_combined.png'), dpi=300, bbox_inches='tight')
    plt.close()

def create_term_comparison_charts(term_df, metrics, clean_metric_names):
    """Create charts comparing different terms"""
    
    # Create a figure with subplots
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.ravel()
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']  # Different colors for each term
    
    for i, metric in enumerate(metrics):
        term_values = []
        term_labels = []
        
        for term in ['Term 1', 'Term 2', 'Term 3']:
            term_data = term_df[term_df['Term'] == term]
            if not term_data.empty:
                term_values.append(term_data[metric].values[0])
                term_labels.append(term)
        
        # Create bar chart
        bars = axes[i].bar(term_labels, term_values, color=colors[:len(term_values)], alpha=0.7)
        
        # Add value labels on bars
        for bar, value in zip(bars, term_values):
            height = bar.get_height()
            axes[i].text(bar.get_x() + bar.get_width()/2., height + 0.05,
                        f'{value:.2f}', ha='center', va='bottom')
        
        axes[i].set_title(clean_metric_names[metric])
        axes[i].set_ylabel('Score (1-10)')
        axes[i].set_ylim(0, 10)
        axes[i].yaxis.set_major_locator(MaxNLocator(integer=True))
    
    plt.suptitle('Wellbeing Metrics: Comparison Across Terms', fontsize=16)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'term_comparison_detailed.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a line chart showing trends across terms
    plt.figure(figsize=(14, 8))
    
    for metric in metrics[:4]:  # Just show first 4 metrics for clarity
        term_values = []
        
        for term in ['Term 1', 'Term 2', 'Term 3']:
            term_data = term_df[term_df['Term'] == term]
            if not term_data.empty:
                term_values.append(term_data[metric].values[0])
        
        if len(term_values) == 3:
            plt.plot(['Term 1', 'Term 2', 'Term 3'], term_values, 
                    marker='o', linewidth=2, markersize=8, label=clean_metric_names[metric])
    
    plt.xlabel('Term')
    plt.ylabel('Average Score')
    plt.title('Wellbeing Trends Across Terms')
    plt.legend()
    plt.ylim(0, 10)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'term_trends.png'), dpi=300, bbox_inches='tight')
    plt.close()

def create_response_count_charts(school_df, term_df):
    """Create charts showing response counts"""
    
    # School level response counts
    school_counts = []
    school_labels = []
    
    for school in ['High School', 'Middle School']:
        count_data = school_df[school_df['SchoolLevel'] == school]['What grade level are you?_count']
        if not count_data.empty:
            count = count_data.values[0]
            school_counts.append(count)
            school_labels.append(school)
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(school_labels, school_counts, color=['#FF6B6B', '#4ECDC4'], alpha=0.7)
    
    # Add value labels on bars
    for bar, count in zip(bars, school_counts):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 5,
                f'{count}', ha='center', va='bottom')
    
    plt.title('Survey Responses by School Level')
    plt.ylabel('Number of Responses')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'response_count_by_school.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Term response counts
    term_counts = []
    term_labels = []
    
    for term in ['Term 1', 'Term 2', 'Term 3']:
        term_data = term_df[term_df['Term'] == term]
        if not term_data.empty:
            count_data = term_data['What grade level are you?_count']
            if not count_data.empty:
                count = count_data.values[0]
                term_counts.append(count)
                term_labels.append(term)
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(term_labels, term_counts, color=['#FF6B6B', '#4ECDC4', '#45B7D1'], alpha=0.7)
    
    # Add value labels on bars
    for bar, count in zip(bars, term_counts):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 5,
                f'{count}', ha='center', va='bottom')
    
    plt.title('Survey Responses by Term')
    plt.ylabel('Number of Responses')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'response_count_by_term.png'), dpi=300, bbox_inches='tight')
    plt.close()

def create_detailed_analysis_charts(school_df, term_df, metrics, clean_metric_names):
    """Create detailed analysis charts"""
    
    # Stress vs Support Analysis
    plt.figure(figsize=(12, 8))
    
    # High School
    hs_data = school_df[school_df['SchoolLevel'] == 'High School'].iloc[0]
    plt.scatter(hs_data['How often do you feel supported by your teachers and advisors?_mean'],
               hs_data['To what extent do you feel the workload is making you feel stressed?_mean'],
               s=300, color='#FF6B6B', alpha=0.7, label='High School')
    
    # Middle School
    ms_data = school_df[school_df['SchoolLevel'] == 'Middle School'].iloc[0]
    plt.scatter(ms_data['How often do you feel supported by your teachers and advisors?_mean'],
               ms_data['To what extent do you feel the workload is making you feel stressed?_mean'],
               s=300, color='#4ECDC4', alpha=0.7, label='Middle School')
    
    plt.xlabel('Teacher Support Score')
    plt.ylabel('Stress Level')
    plt.title('Stress Level vs Teacher Support')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Add school labels
    plt.annotate('High School', 
                (hs_data['How often do you feel supported by your teachers and advisors?_mean'],
                 hs_data['To what extent do you feel the workload is making you feel stressed?_mean']),
                textcoords="offset points", xytext=(0,10), ha='center')
    
    plt.annotate('Middle School', 
                (ms_data['How often do you feel supported by your teachers and advisors?_mean'],
                 ms_data['To what extent do you feel the workload is making you feel stressed?_mean']),
                textcoords="offset points", xytext=(0,10), ha='center')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'stress_vs_support.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Positivity vs Overall Experience
    plt.figure(figsize=(12, 8))
    
    # High School
    plt.scatter(hs_data['How often do you feel positive in school?_mean'],
               hs_data['Overall, how do you feel about the first few months of school?_mean'],
               s=300, color='#FF6B6B', alpha=0.7, label='High School')
    
    # Middle School
    plt.scatter(ms_data['How often do you feel positive in school?_mean'],
               ms_data['Overall, how do you feel about the first few months of school?_mean'],
               s=300, color='#4ECDC4', alpha=0.7, label='Middle School')
    
    plt.xlabel('Positivity Score')
    plt.ylabel('Overall Experience Score')
    plt.title('Overall Experience vs Positivity')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Add school labels
    plt.annotate('High School', 
                (hs_data['How often do you feel positive in school?_mean'],
                 hs_data['Overall, how do you feel about the first few months of school?_mean']),
                textcoords="offset points", xytext=(0,10), ha='center')
    
    plt.annotate('Middle School', 
                (ms_data['How often do you feel positive in school?_mean'],
                 ms_data['Overall, how do you feel about the first few months of school?_mean']),
                textcoords="offset points", xytext=(0,10), ha='center')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'positivity_vs_experience.png'), dpi=300, bbox_inches='tight')
    plt.close()

if __name__ == "__main__":
    create_visualizations()


Creating radar chart...
Creating school level comparison charts...
Creating term comparison charts...
Creating response count charts...
Creating detailed analysis charts...
All visualizations saved to: D:\Wellbeing Survey Project\Summary_Graphs


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import os

# Set up the style
plt.style.use('default')
sns.set_palette("husl")

# Create output directory
output_dir = r"D:\Wellbeing Survey Project\Outlier_Analysis"
os.makedirs(output_dir, exist_ok=True)

def load_and_clean_data(file_path):
    """Load and clean the survey data"""
    print("Loading data...")
    df = pd.read_excel(file_path, sheet_name='Sheet1')
    
    # Clean column names
    df.columns = df.columns.str.strip()
    
    # Define numeric columns
    numeric_cols = [
        'What grade level are you?', 
        'How often do you feel positive in school?', 
        'How often do you feel supported by your teachers and advisors?', 
        'How often do you feel listened to by your teachers and advisors?',
        'How often do you feel listened to and supported by your friends?',
        'How do you feel you are managing the workload?',
        'To what extent do you feel the workload is making you feel stressed?',
        'Overall, how do you feel about the first few months of school?'
    ]
    
    # Convert to numeric, handling errors
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Clean SchoolLevel column
    df['SchoolLevel'] = df['SchoolLevel'].str.strip()
    
    print(f"Data loaded successfully. Shape: {df.shape}")
    return df, numeric_cols

def detect_outliers_zscore(df, numeric_cols, threshold=3):
    """Detect outliers using Z-score method"""
    outliers = {}
    
    for col in numeric_cols:
        # Calculate Z-scores
        z_scores = np.abs(stats.zscore(df[col].dropna()))
        
        # Find outliers
        outlier_indices = np.where(z_scores > threshold)[0]
        outlier_values = df[col].iloc[outlier_indices].values if len(outlier_indices) > 0 else []
        
        outliers[col] = {
            'indices': outlier_indices,
            'values': outlier_values,
            'count': len(outlier_indices),
            'percentage': (len(outlier_indices) / len(df[col].dropna())) * 100
        }
    
    return outliers

def detect_outliers_iqr(df, numeric_cols):
    """Detect outliers using IQR method"""
    outliers = {}
    
    for col in numeric_cols:
        # Calculate IQR
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        # Define outlier bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Find outliers
        outlier_indices = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
        outlier_values = df[col].loc[outlier_indices].values if len(outlier_indices) > 0 else []
        
        outliers[col] = {
            'indices': outlier_indices,
            'values': outlier_values,
            'count': len(outlier_indices),
            'percentage': (len(outlier_indices) / len(df[col].dropna())) * 100,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound
        }
    
    return outliers

def create_outlier_summary(outliers_zscore, outliers_iqr, numeric_cols):
    """Create a summary of outlier detection results"""
    summary_data = []
    
    for col in numeric_cols:
        summary_data.append({
            'Metric': col,
            'Z-Score Outliers': outliers_zscore[col]['count'],
            'Z-Score %': outliers_zscore[col]['percentage'],
            'IQR Outliers': outliers_iqr[col]['count'],
            'IQR %': outliers_iqr[col]['percentage']
        })
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df

def plot_outlier_detection(df, numeric_cols, outliers_iqr, method_name):
    """Create boxplots with outliers highlighted"""
    n_cols = len(numeric_cols)
    n_rows = (n_cols + 3) // 4  # Calculate rows needed for 4 columns
    
    fig, axes = plt.subplots(n_rows, 4, figsize=(20, 5 * n_rows))
    axes = axes.ravel()  # Flatten the axes array
    
    for i, col in enumerate(numeric_cols):
        if i < len(axes):  # Ensure we don't exceed axes count
            # Create boxplot
            bp = axes[i].boxplot(df[col].dropna(), patch_artist=True)
            
            # Style the boxplot
            bp['boxes'][0].set_facecolor('lightblue')
            bp['medians'][0].set_color('red')
            
            # Highlight outliers
            outlier_values = outliers_iqr[col]['values']
            if len(outlier_values) > 0:
                outlier_positions = [1] * len(outlier_values)  # All at x=1 for single boxplot
                axes[i].scatter(outlier_positions, outlier_values, color='red', s=50, alpha=0.6, zorder=10)
            
            axes[i].set_title(f'{col}\n({len(outlier_values)} outliers)')
            axes[i].set_ylabel('Score')
            
            # Remove x-axis labels for cleaner look
            axes[i].set_xticklabels([])
    
    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)
    
    plt.suptitle(f'Outlier Detection using {method_name} Method', fontsize=16)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'outlier_boxplots_{method_name.lower()}.png'), 
                dpi=300, bbox_inches='tight')
    plt.close()

def plot_outlier_distribution(df, numeric_cols, outliers_iqr):
    """Create distribution plots with outliers highlighted"""
    n_cols = len(numeric_cols)
    n_rows = (n_cols + 3) // 4
    
    fig, axes = plt.subplots(n_rows, 4, figsize=(20, 5 * n_rows))
    axes = axes.ravel()
    
    for i, col in enumerate(numeric_cols):
        if i < len(axes):
            # Plot distribution
            axes[i].hist(df[col].dropna(), bins=20, alpha=0.7, color='lightblue', edgecolor='black')
            
            # Highlight outliers
            outlier_values = outliers_iqr[col]['values']
            if len(outlier_values) > 0:
                # Add red lines for outliers
                for value in outlier_values:
                    axes[i].axvline(value, color='red', linestyle='--', alpha=0.7)
            
            axes[i].set_title(f'{col}\n({len(outlier_values)} outliers)')
            axes[i].set_xlabel('Score')
            axes[i].set_ylabel('Frequency')
    
    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)
    
    plt.suptitle('Distribution of Scores with Outliers Highlighted', fontsize=16)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'outlier_distribution.png'), dpi=300, bbox_inches='tight')
    plt.close()

def analyze_outliers_by_group(df, numeric_cols, outliers_iqr):
    """Analyze outliers by school level and term"""
    results = {}
    
    for col in numeric_cols:
        outlier_indices = outliers_iqr[col]['indices']
        outlier_data = df.loc[outlier_indices]
        
        # Analyze by school level
        school_level_counts = outlier_data['SchoolLevel'].value_counts()
        school_level_percentage = (school_level_counts / len(outlier_data)) * 100
        
        # Analyze by term
        term_counts = outlier_data['Term'].value_counts()
        term_percentage = (term_counts / len(outlier_data)) * 100
        
        results[col] = {
            'school_level': school_level_counts.to_dict(),
            'school_level_percentage': school_level_percentage.to_dict(),
            'term': term_counts.to_dict(),
            'term_percentage': term_percentage.to_dict()
        }
    
    return results

def plot_outlier_demographics(outlier_analysis, numeric_cols):
    """Create demographic analysis of outliers"""
    # School level analysis
    school_data = []
    for col in numeric_cols:
        for school, count in outlier_analysis[col]['school_level'].items():
            school_data.append({
                'Metric': col,
                'School Level': school,
                'Outlier Count': count,
                'Percentage': outlier_analysis[col]['school_level_percentage'][school]
            })
    
    school_df = pd.DataFrame(school_data)
    
    # Pivot for plotting
    school_pivot = school_df.pivot_table(index='School Level', columns='Metric', 
                                       values='Outlier Count', aggfunc='sum', fill_value=0)
    
    plt.figure(figsize=(15, 8))
    school_pivot.plot(kind='bar', stacked=True)
    plt.title('Outlier Distribution by School Level')
    plt.ylabel('Number of Outliers')
    plt.xticks(rotation=45)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'outliers_by_school_level.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Term analysis
    term_data = []
    for col in numeric_cols:
        for term, count in outlier_analysis[col]['term'].items():
            term_data.append({
                'Metric': col,
                'Term': term,
                'Outlier Count': count,
                'Percentage': outlier_analysis[col]['term_percentage'][term]
            })
    
    term_df = pd.DataFrame(term_data)
    
    # Pivot for plotting
    term_pivot = term_df.pivot_table(index='Term', columns='Metric', 
                                   values='Outlier Count', aggfunc='sum', fill_value=0)
    
    plt.figure(figsize=(15, 8))
    term_pivot.plot(kind='bar', stacked=True)
    plt.title('Outlier Distribution by Term')
    plt.ylabel('Number of Outliers')
    plt.xticks(rotation=45)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'outliers_by_term.png'), dpi=300, bbox_inches='tight')
    plt.close()

def create_detailed_outlier_report(df, outliers_iqr, numeric_cols):
    """Create a detailed report of all outliers"""
    outlier_records = []
    
    for col in numeric_cols:
        outlier_indices = outliers_iqr[col]['indices']
        for idx in outlier_indices:
            record = df.loc[idx].to_dict()
            record['Outlier_Metric'] = col
            record['Outlier_Value'] = df.loc[idx, col]
            record['IQR_Lower_Bound'] = outliers_iqr[col]['lower_bound']
            record['IQR_Upper_Bound'] = outliers_iqr[col]['upper_bound']
            outlier_records.append(record)
    
    outlier_df = pd.DataFrame(outlier_records)
    
    # Save to Excel
    outlier_df.to_excel(os.path.join(output_dir, 'detailed_outlier_report.xlsx'), index=False)
    
    return outlier_df

def main():
    """Main function to run outlier analysis"""
    file_path = r"D:\Wellbeing Survey Project\combined_terms.xlsx"
    
    # Load and clean data
    df, numeric_cols = load_and_clean_data(file_path)
    
    print("Detecting outliers using Z-score method...")
    outliers_zscore = detect_outliers_zscore(df, numeric_cols)
    
    print("Detecting outliers using IQR method...")
    outliers_iqr = detect_outliers_iqr(df, numeric_cols)
    
    print("Creating outlier summary...")
    outlier_summary = create_outlier_summary(outliers_zscore, outliers_iqr, numeric_cols)
    outlier_summary.to_excel(os.path.join(output_dir, 'outlier_summary.xlsx'), index=False)
    
    print("Creating visualizations...")
    plot_outlier_detection(df, numeric_cols, outliers_iqr, "IQR")
    plot_outlier_distribution(df, numeric_cols, outliers_iqr)
    
    print("Analyzing outlier demographics...")
    outlier_analysis = analyze_outliers_by_group(df, numeric_cols, outliers_iqr)
    plot_outlier_demographics(outlier_analysis, numeric_cols)
    
    print("Creating detailed outlier report...")
    detailed_report = create_detailed_outlier_report(df, outliers_iqr, numeric_cols)
    
    print("\n=== OUTLIER ANALYSIS SUMMARY ===")
    print(f"Total records: {len(df)}")
    print("\nOutlier counts by metric (IQR method):")
    for col in numeric_cols:
        count = outliers_iqr[col]['count']
        percentage = outliers_iqr[col]['percentage']
        print(f"{col}: {count} outliers ({percentage:.1f}%)")
    
    print(f"\nAll analysis results saved to: {output_dir}")

if __name__ == "__main__":
    main()

Loading data...
Data loaded successfully. Shape: (480, 13)
Detecting outliers using Z-score method...
Detecting outliers using IQR method...
Creating outlier summary...
Creating visualizations...
Analyzing outlier demographics...
Creating detailed outlier report...

=== OUTLIER ANALYSIS SUMMARY ===
Total records: 480

Outlier counts by metric (IQR method):
What grade level are you?: 0 outliers (0.0%)
How often do you feel positive in school?: 85 outliers (18.0%)
How often do you feel supported by your teachers and advisors?: 8 outliers (1.7%)
How often do you feel listened to by your teachers and advisors?: 10 outliers (2.1%)
How often do you feel listened to and supported by your friends?: 9 outliers (1.9%)
How do you feel you are managing the workload?: 14 outliers (3.0%)
To what extent do you feel the workload is making you feel stressed?: 0 outliers (0.0%)
Overall, how do you feel about the first few months of school?: 5 outliers (1.0%)

All analysis results saved to: D:\Wellbeing 

<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import os

# Set up the style
plt.style.use('default')
sns.set_style("whitegrid")
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

# Create output directory
output_dir = r"D:\Wellbeing Survey Project\Outlier_Analysis"
os.makedirs(output_dir, exist_ok=True)

def create_enhanced_outlier_charts(df, numeric_cols, outliers_iqr):
    """Create enhanced and better-looking outlier charts"""
    
    # Clean metric names for display
    clean_metric_names = {
        'What grade level are you?': 'Grade Level',
        'How often do you feel positive in school?': 'Positivity',
        'How often do you feel supported by your teachers and advisors?': 'Teacher Support',
        'How often do you feel listened to by your teachers and advisors?': 'Listened To (Teachers)',
        'How often do you feel listened to and supported by your friends?': 'Friend Support',
        'How do you feel you are managing the workload?': 'Workload Management',
        'To what extent do you feel the workload is making you feel stressed?': 'Stress Level',
        'Overall, how do you feel about the first few months of school?': 'Overall Experience'
    }
    
    # 1. Enhanced Outlier Analysis by Term
    print("Creating enhanced outlier by terms chart...")
    create_enhanced_term_chart(df, numeric_cols, outliers_iqr, clean_metric_names)
    
    # 2. Enhanced School Level Comparison
    print("Creating enhanced school level chart...")
    create_enhanced_school_chart(df, numeric_cols, outliers_iqr, clean_metric_names)
    
    # 3. Outlier Percentage Comparison
    print("Creating outlier percentage chart...")
    create_outlier_percentage_chart(df, numeric_cols, outliers_iqr, clean_metric_names)

def create_enhanced_term_chart(df, numeric_cols, outliers_iqr, clean_metric_names):
    """Create a visually enhanced outlier analysis by term"""
    
    # Prepare data for term analysis
    term_data = []
    for col in numeric_cols:
        outlier_indices = outliers_iqr[col]['indices']
        outlier_df = df.loc[outlier_indices]
        
        term_counts = outlier_df['Term'].value_counts().to_dict()
        total_outliers = len(outlier_indices)
        
        for term, count in term_counts.items():
            term_data.append({
                'Metric': clean_metric_names[col],
                'Term': term,
                'Outlier Count': count,
                'Total Outliers': total_outliers,
                'Percentage': (count / total_outliers) * 100 if total_outliers > 0 else 0
            })
    
    term_df = pd.DataFrame(term_data)
    
    # Create the visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12), gridspec_kw={'height_ratios': [2, 1]})
    
    # Color palette
    term_colors = {'Term 1': '#FF6B6B', 'Term 2': '#4ECDC4', 'Term 3': '#45B7D1'}
    metrics = term_df['Metric'].unique()
    
    # Bar positions
    x = np.arange(len(metrics))
    width = 0.25
    terms = sorted(term_df['Term'].unique())
    
    # Plot stacked bars
    bottom = np.zeros(len(metrics))
    for i, term in enumerate(terms):
        term_values = []
        for metric in metrics:
            value = term_df[(term_df['Metric'] == metric) & (term_df['Term'] == term)]['Outlier Count'].sum()
            term_values.append(value)
        
        bars = ax1.bar(x + i*width, term_values, width, label=term, 
                      color=term_colors[term], alpha=0.8, edgecolor='white', linewidth=1)
        
        # Add value labels
        for j, value in enumerate(term_values):
            if value > 0:
                ax1.text(x[j] + i*width, bottom[j] + value/2, f'{value}', 
                        ha='center', va='center', fontweight='bold', color='white', fontsize=9)
        
        bottom += np.array(term_values)
    
    # Customize the main chart
    ax1.set_xlabel('Wellbeing Metrics', fontweight='bold')
    ax1.set_ylabel('Number of Outliers', fontweight='bold')
    ax1.set_title('Outlier Distribution by Term and Metric\n(IQR Method)', fontsize=16, fontweight='bold', pad=20)
    ax1.set_xticks(x + width)
    ax1.set_xticklabels(metrics, rotation=45, ha='right')
    ax1.legend(title='Term', title_fontsize=12, fontsize=11)
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Add some styling
    for spine in ax1.spines.values():
        spine.set_visible(False)
    
    # Create percentage subplot
    percentage_data = []
    for metric in metrics:
        metric_total = term_df[term_df['Metric'] == metric]['Outlier Count'].sum()
        metric_data = term_df[term_df['Metric'] == metric]
        for term in terms:
            term_count = metric_data[metric_data['Term'] == term]['Outlier Count'].sum()
            percentage = (term_count / metric_total * 100) if metric_total > 0 else 0
            percentage_data.append({'Metric': metric, 'Term': term, 'Percentage': percentage})
    
    percentage_df = pd.DataFrame(percentage_data)
    
    # Plot percentage heatmap
    pivot_data = percentage_df.pivot(index='Metric', columns='Term', values='Percentage')
    im = ax2.imshow(pivot_data.values, cmap='YlOrRd', aspect='auto', vmin=0, vmax=100)
    
    # Customize heatmap
    ax2.set_xticks(np.arange(len(terms)))
    ax2.set_xticklabels(terms)
    ax2.set_yticks(np.arange(len(metrics)))
    ax2.set_yticklabels(metrics)
    ax2.set_title('Outlier Percentage Distribution by Term', fontsize=14, fontweight='bold', pad=15)
    
    # Add percentage values to heatmap
    for i in range(len(metrics)):
        for j in range(len(terms)):
            value = pivot_data.iloc[i, j]
            if not np.isnan(value) and value > 0:
                text = ax2.text(j, i, f'{value:.1f}%', ha='center', va='center', 
                               fontweight='bold', color='white' if value > 50 else 'black')
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax2)
    cbar.set_label('Percentage (%)', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'enhanced_outliers_by_term.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a separate normalized stacked bar chart
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Prepare normalized data
    normalized_data = []
    for metric in metrics:
        metric_total = term_df[term_df['Metric'] == metric]['Outlier Count'].sum()
        if metric_total > 0:
            for term in terms:
                term_count = term_df[(term_df['Metric'] == metric) & (term_df['Term'] == term)]['Outlier Count'].sum()
                normalized_data.append({
                    'Metric': metric,
                    'Term': term,
                    'Percentage': (term_count / metric_total) * 100
                })
    
    normalized_df = pd.DataFrame(normalized_data)
    pivot_normalized = normalized_df.pivot(index='Metric', columns='Term', values='Percentage').fillna(0)
    
    # Plot normalized stacked bars
    bottom = np.zeros(len(metrics))
    for i, term in enumerate(terms):
        values = pivot_normalized[term].values
        bars = ax.bar(range(len(metrics)), values, bottom=bottom, label=term,
                     color=term_colors[term], alpha=0.8, edgecolor='white', linewidth=1)
        bottom += values
    
    # Customize normalized chart
    ax.set_xlabel('Wellbeing Metrics', fontweight='bold')
    ax.set_ylabel('Percentage of Outliers (%)', fontweight='bold')
    ax.set_title('Normalized Outlier Distribution by Term\n(Percentage of Total Outliers per Metric)', 
                fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(range(len(metrics)))
    ax.set_xticklabels(metrics, rotation=45, ha='right')
    ax.legend(title='Term', title_fontsize=11, fontsize=10)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add percentage labels
    for i, metric in enumerate(metrics):
        total = 0
        for term in terms:
            value = pivot_normalized.loc[metric, term]
            if value > 5:  # Only label significant segments
                ax.text(i, total + value/2, f'{value:.1f}%', ha='center', va='center',
                       fontweight='bold', color='white', fontsize=8)
            total += value
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'normalized_outliers_by_term.png'), dpi=300, bbox_inches='tight')
    plt.close()

def create_enhanced_school_chart(df, numeric_cols, outliers_iqr, clean_metric_names):
    """Create enhanced school level outlier chart"""
    
    school_data = []
    for col in numeric_cols:
        outlier_indices = outliers_iqr[col]['indices']
        outlier_df = df.loc[outlier_indices]
        
        school_counts = outlier_df['SchoolLevel'].value_counts().to_dict()
        
        for school, count in school_counts.items():
            school_data.append({
                'Metric': clean_metric_names[col],
                'School Level': school,
                'Outlier Count': count
            })
    
    school_df = pd.DataFrame(school_data)
    
    # Create visualization
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Color palette
    school_colors = {'High School': '#FF6B6B', 'Middle School': '#4ECDC4'}
    metrics = school_df['Metric'].unique()
    
    # Bar positions
    x = np.arange(len(metrics))
    width = 0.35
    
    # Plot grouped bars
    for i, school in enumerate(['High School', 'Middle School']):
        values = []
        for metric in metrics:
            count = school_df[(school_df['Metric'] == metric) & 
                             (school_df['School Level'] == school)]['Outlier Count'].sum()
            values.append(count)
        
        bars = ax.bar(x + i*width, values, width, label=school, 
                     color=school_colors[school], alpha=0.8, edgecolor='white', linewidth=1)
        
        # Add value labels
        for j, value in enumerate(values):
            if value > 0:
                ax.text(x[j] + i*width, value + 0.5, f'{value}', 
                       ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # Customize chart
    ax.set_xlabel('Wellbeing Metrics', fontweight='bold')
    ax.set_ylabel('Number of Outliers', fontweight='bold')
    ax.set_title('Outlier Distribution by School Level\n(IQR Method)', fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x + width/2)
    ax.set_xticklabels(metrics, rotation=45, ha='right')
    ax.legend(title='School Level', title_fontsize=11, fontsize=10)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'enhanced_outliers_by_school.png'), dpi=300, bbox_inches='tight')
    plt.close()

def create_outlier_percentage_chart(df, numeric_cols, outliers_iqr, clean_metric_names):
    """Create outlier percentage comparison chart"""
    
    percentage_data = []
    for col in numeric_cols:
        total_responses = len(df[col].dropna())
        outlier_count = outliers_iqr[col]['count']
        percentage = (outlier_count / total_responses) * 100 if total_responses > 0 else 0
        
        percentage_data.append({
            'Metric': clean_metric_names[col],
            'Outlier Percentage': percentage,
            'Outlier Count': outlier_count,
            'Total Responses': total_responses
        })
    
    percentage_df = pd.DataFrame(percentage_data)
    percentage_df = percentage_df.sort_values('Outlier Percentage', ascending=False)
    
    # Create visualization
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Color based on percentage
    colors = ['#FF6B6B' if p > 10 else '#4ECDC4' if p > 5 else '#45B7D1' 
             for p in percentage_df['Outlier Percentage']]
    
    bars = ax.barh(percentage_df['Metric'], percentage_df['Outlier Percentage'], 
                  color=colors, alpha=0.8, edgecolor='white', linewidth=1)
    
    # Add value labels
    for i, (percentage, count, total) in enumerate(zip(
        percentage_df['Outlier Percentage'], 
        percentage_df['Outlier Count'], 
        percentage_df['Total Responses']
    )):
        ax.text(percentage + 0.5, i, f'{percentage:.1f}% ({count}/{total})', 
               va='center', fontweight='bold', fontsize=9)
    
    # Customize chart
    ax.set_xlabel('Outlier Percentage (%)', fontweight='bold')
    ax.set_ylabel('Wellbeing Metrics', fontweight='bold')
    ax.set_title('Outlier Percentage by Metric\n(IQR Method)', fontsize=14, fontweight='bold', pad=20)
    ax.grid(True, alpha=0.3, axis='x')
    ax.set_xlim(0, percentage_df['Outlier Percentage'].max() * 1.1)
    
    # Add some styling
    for spine in ax.spines.values():
        spine.set_visible(False)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'outlier_percentage_comparison.png'), dpi=300, bbox_inches='tight')
    plt.close()

# Add this function to your main analysis
def enhanced_main():
    """Enhanced main function with better visualizations"""
    file_path = r"D:\Wellbeing Survey Project\combined_terms.xlsx"
    
    # Load and clean data
    df, numeric_cols = load_and_clean_data(file_path)
    
    print("Detecting outliers using IQR method...")
    outliers_iqr = detect_outliers_iqr(df, numeric_cols)
    
    print("Creating enhanced visualizations...")
    create_enhanced_outlier_charts(df, numeric_cols, outliers_iqr)
    
    print(f"Enhanced visualizations saved to: {output_dir}")

# You'll need to include these helper functions from the previous script:
def load_and_clean_data(file_path):
    """Load and clean the survey data"""
    print("Loading data...")
    df = pd.read_excel(file_path, sheet_name='Sheet1')
    
    # Clean column names
    df.columns = df.columns.str.strip()
    
    # Define numeric columns
    numeric_cols = [
        'What grade level are you?', 
        'How often do you feel positive in school?', 
        'How often do you feel supported by your teachers and advisors?', 
        'How often do you feel listened to by your teachers and advisors?',
        'How often do you feel listened to and supported by your friends?',
        'How do you feel you are managing the workload?',
        'To what extent do you feel the workload is making you feel stressed?',
        'Overall, how do you feel about the first few months of school?'
    ]
    
    # Convert to numeric, handling errors
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Clean SchoolLevel column
    df['SchoolLevel'] = df['SchoolLevel'].str.strip()
    
    print(f"Data loaded successfully. Shape: {df.shape}")
    return df, numeric_cols

def detect_outliers_iqr(df, numeric_cols):
    """Detect outliers using IQR method"""
    outliers = {}
    
    for col in numeric_cols:
        # Calculate IQR
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        # Define outlier bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Find outliers
        outlier_indices = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
        outlier_values = df[col].loc[outlier_indices].values if len(outlier_indices) > 0 else []
        
        outliers[col] = {
            'indices': outlier_indices,
            'values': outlier_values,
            'count': len(outlier_indices),
            'percentage': (len(outlier_indices) / len(df[col].dropna())) * 100,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound
        }
    
    return outliers

if __name__ == "__main__":
    enhanced_main()

Loading data...
Data loaded successfully. Shape: (480, 13)
Detecting outliers using IQR method...
Creating enhanced visualizations...
Creating enhanced outlier by terms chart...
Creating enhanced school level chart...
Creating outlier percentage chart...
Enhanced visualizations saved to: D:\Wellbeing Survey Project\Outlier_Analysis
